In [ ]:
!pip install mediapipe opencv-python

     |████████████████████████████████| 30.4 MB 69 kB/s 


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

In [ ]:
from google.colab import drive # 국방부 해커톤/input에 있는 영상 사본 만들고 사본을 옮겨서 이름 바꾸기 / 원본 안옮기게 주의!
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# **Finding XYZ**

In [ ]:
def find_xyz(ind_list, landmark):
  a = landmark[ind_list[0]]
  b = landmark[ind_list[1]]
  c = landmark[ind_list[2]]

  first = [a.x,a.y,a.z]
  mid = [b.x,b.y,b.z]
  end = [c.x,c.y,c.z]
  return first, mid, end

# **Calculating Angle**

In [ ]:
def calculate_angle3D(a,b,c,direction):
  """
  calculate_angle3D is divided by left and right side because this function uses external product
  input : a,b,c -> landmarks with shape [x,y,z,visibility]
          direction -> int -1 or 1
                      -1 means Video(photo) for a person's left side and 1 means Video(photo) for a person's right side
  output : angle between vector ba and bc with range 0~360
  """
  # external product's z value
  external_z = (b[0]-a[0])*(b[1]-c[1]) - (b[1]-a[1])*(b[0]-c[0])

  a = np.array(a) #first
  b = np.array(b) #mid
  c = np.array(c) #end

  ba = b-a
  bc = b-c
  dot_result = np.dot(ba, bc)


  ba_size = np.linalg.norm(ba)
  bc_size = np.linalg.norm(bc)
  radi = np.arccos(dot_result / (ba_size*bc_size))
  angle = np.abs(radi*180.0/np.pi)

  # left side
  if external_z * direction > 0:
    angle = 360 - angle

  return angle


# **Checking Angle(xyz)**

In [ ]:
video_path = "/content/wrong/6.mp4"
cap = cv2.VideoCapture(video_path)
elbow_state = 'up'


#우측 관절값
joint_indx = {'check_right_elbow':[16,14,12],'check_right_hip':[12,24,26],'check_right_knee':[24,26,28]} 

"""
temp_w6은 팔굽혀펴기 운동을 한번 할떄마다(내려갔다 올라올 때마다) 해당 관절 부분의 각도를 최신화함. 이전에 했던 팔굽혀펴기의 각도값들은 다 초기화함.
      -> 앱 상에서 관절 각도값을 저장하는 데에 무리가 가지않게 list를 초기화시켜줌.
"""
temp_w6 = {'check_right_elbow':[],'check_right_hip':[],'check_right_knee':[]}
pushups = []

"""
pushups는 list(list)형식이다. 안쪽 list는 length가 4이며 각각 Is_elbow_up, Is_elbow_down, hip_condition, knee_condition을 의미한다.
Is_elbow_up -> int 0 or 1 -> 팔꿈치를 완전히 펴면 1, 완전히 펴지 않으면 0
Is_elbow_down -> int 0 or 1 -> 팔꿈치를 완전히 굽히면 1, 완전히 굽히지 않으면 0
hip_condition -> int 0 or 1 or 2 -> 골반이 정상이면 0, 너무 내려가면 1, 너무 올라가면 2
knee_condition -> int 1 or 0 -> 무릎 각도가 정상범위면 1, 너무 내려가면 0 
예를 들어, pushup[0]이 [1,0,1,1]이면 첫번째 푸쉬업 동작이 올라올 때 팔꿈치를 완전히 폈고, 내려갈때 완전히 팔을 굽히지는 않았으며, 골반은 과도하게 내려갔고, 무릎각도는 정상범위라는 의미이다.
"""

if cap.isOpened():
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    res=(int(width), int(height))
    fourcc = cv2.VideoWriter_fourcc(*'MP4V') #codec
    out = cv2.VideoWriter('wtemp6.mp4', fourcc, 20.0, res)

    frame = None
    while True:
      try:
        ret, frame = cap.read()
      except cv2.error:
        continue
      if not ret:
        break

      image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
      results = pose.process(image)

      if results.pose_landmarks:
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        for id, im in enumerate(results.pose_landmarks.landmark):
          h, w, c = frame.shape
          cx, cy = int(im.x*w), int(im.y*h)
          cv2.circle(frame, (cx,cy), 5, (255, 0, 0), cv2.FILLED)
        landmark = results.pose_landmarks.landmark
        for key,val in joint_indx.items():
          first,mid,end = find_xyz(val, landmark)
          angle = calculate_angle3D(first,mid,end, 1) #각도 계산
          mid = mid[0:2]
          cv2.putText(frame, str(angle),tuple(np.multiply(mid, [2400,1080]).astype(int)),cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255,0,0),5,cv2.LINE_AA)
          temp_w6[key].append(angle)
 

        elbow_angle = temp_w6['check_right_elbow'][-1]
        if elbow_angle > 135 and elbow_state == 'down':
            elbow_state = 'up'
            pushups.append([])
            # 팔꿈치를 핀 정도가 160도보다 큰 경우
            if max(temp_w6['check_right_elbow']) > 160:
              pushups[-1].append(1)
            # 팔꿈치를 덜 핀 경우
            else:
              pushups[-1].append(0)
            # 팔꿈치를 굽힌 정도가 90도보다 작은 경우

            if min(temp_w6['check_right_elbow']) < 90:
              pushups[-1].append(1)
            # 팔꿈치를 덜 굽힌 경우
            else:
              pushups[-1].append(0)
            # 팔꿈치 각도 데이터 초기화 (어플에서의 변수 저장 값이 많아지는 것을 방지하기 위해 팔굽 하나 할때마다 각도 축적해놓은거를 초기화함.)
            temp_w6['check_right_elbow'] = []

            # 팔꿈치 굽혔다 필 동안 골반이 160도보다 내려갔는지(골반이 너무 아래로 내려갔는지)
            if min(temp_w6['check_right_hip']) < 160:
              pushups[-1].append(1)
            # 팔꿈치 굽혔다 필 동안 골반이 220도보다 올라갔는지(골반이 너무 위로 올라갔는지)
            elif max(temp_w6['check_right_hip']) > 220:
              pushups[-1].append(2)
            # 팔굽하는 동안 골반이 정상 자세(160~220도 사이값)인 경우
            else:
              pushups[-1].append(0)
            temp_w6['check_right_hip'] = []


            if min(temp_w6['check_right_knee']) < 152:
              pushups[-1].append(0)
            else:
              pushups[-1].append(1)
            temp_w6['check_right_knee'] = []

        if elbow_angle < 130 and elbow_state == 'up':
            elbow_state = 'down'

        cv2.putText(frame, str(len(pushups)), (200,300), cv2.FONT_HERSHEY_PLAIN, 4, (255,255,255),2, cv2.LINE_AA)

      out.write(frame)
    out.release()



In [ ]:
pushups

[[1, 0, 1, 0],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 0, 1, 1],
 [1, 0, 1, 0],
 [1, 0, 1, 0],
 [0, 0, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 2, 1]]

#**Evaulate Pushups**

In [ ]:
def pushups_to_score(pushups):
  each_score = []
  for e in pushups:
    Is_elbow_up = e[0]
    Is_elbow_down = e[1]
    Is_hip_good = 0
    if e[2] == 0:
      Is_hip_good = 1
    Is_knee_good = e[3]
    each_score.append(Is_elbow_up*30+Is_elbow_down*30+Is_hip_good*30+Is_knee_good*10)
  return each_score

In [ ]:
# pushups score for each reptition
pushups_to_score(pushups)

[30, 70, 70, 40, 30, 30, 10, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70]

In [ ]:
#total_score
sum(pushups_to_score(pushups))

1050

In [ ]:
from google.colab import files
files.download('wtemp6.mp4')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#**Checking Conditions for Squat**

In [ ]:
def get_distance(lm_from, lm_to):
  x2 = (lm_from.x - lm_to.x)**2
  y2 = (lm_from.y-lm_to.y)**2
  return (x2+y2)**0.5

In [ ]:
video_path = "/content/squat/4.mp4"
cap = cv2.VideoCapture(video_path)
squat_state = 'up'
Is_knee_out = False

#우측 관절값
joint_indx = {'check_right_hip':[12,24,26],'check_right_knee':[24,26,28]} 

"""
temp_squat은 스쿼트 운동을 한번 할떄마다(내려갔다 올라올 때마다) 해당 관절 부분의 각도를 최신화함. 이전에 했던 스쿼트의 각도값들은 다 초기화함.
      -> 앱 상에서 관절 각도값을 저장하는 데에 무리가 가지않게 list를 초기화시켜줌.
"""
temp_squat = {'check_right_hip':[],'check_right_knee':[], 'average_hip_knee' : []} 
squats = []

"""
squats는 list(list)형식이다. 안쪽 list는 length가 4이며 각각 Is_relaxation, Is_contraction, Hip_knee_relation, Is_knee_in을 의미한다.
Is_relaxation -> int 0 or 1 -> hip과 무릎을 완전히 펴면 1, 완전히 펴지 않으면 0
Is_contraction -> int 0 or 1 -> hip과 무릎을 완전히 굽히면 1, 완전히 굽히지 않으면 0
Hip_knee_relation -> int 0 or 1 or 2 -> 스쿼트 시 엉덩이가 내려가면서 무릎을 굽히면(운동 시 엉덩이의 값과 무릎의 각도의 평균값이 180도와 비슷하면) 0, 무릎을 굽히지 않고 엉덩이를 뒤로 빼면 1,
                                        엉덩이를 뒤로 빼지 않고 무릎만 굽히면 2
Is_knee_in -> int 1 or 0 -> 무릎이 발끝(엄지발가락)을 넘지 않으면1, 넘어가면 0 
예를 들어, squats[0]이 [1,0,2,1]이면 첫번째 스쿼트 동작이 올라올 때 완전히 이완을 했고, 내려갈때 완전히 수축하진 않았으며(엉덩이를 더 내려야함),
엉덩이를 빼기보단 무릎만 굽혔고, 무릎이 발끝을 넘어가지 않았다는 의미이다.
"""

if cap.isOpened():
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    res=(int(width), int(height))
    fourcc = cv2.VideoWriter_fourcc(*'MP4V') #codec
    out = cv2.VideoWriter('4out.mp4', fourcc, 20.0, res)

    frame = None
    while True:
      try:
        ret, frame = cap.read()
      except cv2.error:
        continue
      if not ret:
        break

      image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
      results = pose.process(image)

      if results.pose_landmarks:
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        for id, im in enumerate(results.pose_landmarks.landmark):
          h, w, c = frame.shape
          cx, cy = int(im.x*w), int(im.y*h)
          cv2.circle(frame, (cx,cy), 5, (255, 0, 0), cv2.FILLED)
        landmark = results.pose_landmarks.landmark
        for key,val in joint_indx.items():
          first,mid,end = find_xyz(val, landmark)
          angle = calculate_angle3D(first,mid,end, 1) #각도 계산
          mid = mid[0:2]
          cv2.putText(frame, str(angle),tuple(np.multiply(mid, [2400,1080]).astype(int)),cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255,0,0),5,cv2.LINE_AA)
          temp_squat[key].append(angle)
        knee_x = landmark[26].x # knee
        toe_x = landmark[32].x # toe

        # foot의 길이 측정
        foot_length = get_distance(landmark[32], landmark[30])
        # 무릎이 발끝보다 조금 나간거는 허용해줌(스쿼트 특성상 정확히 무릎이 발끝보다 덜 나가지 않음. 또한 오차값도 고려해서 발길이의 19.5%는 threshold값으로 설정함)
        if toe_x + foot_length*0.195 < knee_x:
           Is_knee_out = True

        hip_angle = temp_squat['check_right_hip'][-1]
        knee_angle = temp_squat['check_right_knee'][-1]
        temp_squat['average_hip_knee'].append((hip_angle + knee_angle) / 2)

        
        
        if hip_angle < 235 and knee_angle > 130 and squat_state == 'down':
            squat_state = 'up'
            element = []
            # Is_relaxation
            # 엉덩이를 핀 정도가 195도보다 작은 경우(완전히 이완)
            if min(temp_squat['check_right_hip']) < 195:
              element.append(1)
            # 엉덩이를 덜 핀 경우
            else:
              element.append(0)

            # Is_contraction
            # 엉덩이를 굽힌 정도가 270도보다 큰 경우(완전히 수축)
            if max(temp_squat['check_right_hip']) > 270:
              element.append(1)
            # 엉덩이를 덜 굽힌 경우
            else:
              element.append(0)

            # 엉덩이 각도 데이터 초기화 (어플에서의 변수 저장 값이 많아지는 것을 방지하기 위해 스쿼트 하나 할때마다 각도 축적해놓은거를 초기화함.)
            temp_squat['check_right_hip'] = []
            temp_squat['check_right_knee'] = []


            # Hip_knee_relation
            # 스쿼트 동작동안 엉덩이와 무릎이 균형있게 내려가지 않고 엉덩이가 우선적으로 내려갔는지
            if max(temp_squat['average_hip_knee']) > 193:
              element.append(1)
            # 스쿼트 동작동안 엉덩이와 무릎이 균형있게 내려가지 않고 무릎이 우선적으로 내려갔는지
            elif min(temp_squat['average_hip_knee']) < 176:
              element.append(2)
            # 스쿼트 동작동안 엉덩이와 무릎이 균형있게 내려간 경우 (둘의 각도 평균값이 176~193의 경우)
            else:
              element.append(0)
            temp_squat['average_hip_knee'] = []

            # Is_knee_in
            # 무릎이 발끝보다 안쪽에 있는지 유무
            if Is_knee_out:
              element.append(0)
            else:
              element.append(1)
            
            squats.append(element)
            Is_knee_out = False

        if hip_angle > 245 and knee_angle < 130 and squat_state == 'up':
            squat_state = 'down'

        cv2.putText(frame, str(len(squats)), (200,300), cv2.FONT_HERSHEY_PLAIN, 4, (255,255,255),2, cv2.LINE_AA)

      out.write(frame)
    out.release()



In [ ]:
squats

[[1, 1, 0, 1], [1, 0, 0, 1], [1, 1, 0, 1], [1, 1, 2, 1]]

#**Evaulate Squats**

In [ ]:
def squats_to_score(squats):
  each_score = []
  for e in squats:
    Is_relaxation = e[0]
    Is_contraction = e[1]
    Is_hip_knee_good = 0
    if e[2] == 0:
      Is_hip_knee_good = 1
    Is_knee_in = e[3]
    each_score.append(Is_relaxation*10+Is_contraction*20+Is_hip_knee_good*55+Is_knee_in*15)
  return each_score

In [ ]:
# squats score for each reptition
squats_to_score(squats)

[100, 80, 100, 45]

In [ ]:
sum(squats_to_score(squats))

325

In [ ]:
from google.colab import files
files.download('4out.mp4')
